### Calculate capacity of wind farms by floating/grounded

In [1]:
import pandas as pd
import geopandas as gpd 
import numpy as np 

Get existing capacities

In [13]:
NODES_TO_BE_REMOVED = ["Utsira Nord", "Sørlige Nordsjø I", "Sørlige Nordsjø II"]
OFFSHORE_TECHS = ["Wind_offshr_floating", "Wind_offshr_grounded"]

EMPIRE_capacity = pd.read_excel("Data/Generator.xlsx", sheet_name="MaxInstalledCapacity", header=2)
EMPIRE_capacity_ow = EMPIRE_capacity[EMPIRE_capacity["GeneratorTechnology"].isin(OFFSHORE_TECHS)]
EMPIRE_capacity_ow = EMPIRE_capacity_ow[~EMPIRE_capacity_ow["Node"].isin(NODES_TO_BE_REMOVED)]
EMPIRE_capacity_ow = EMPIRE_capacity_ow.drop(columns=["Unnamed: 3", "Unnamed: 4", "NREAP (2020 number + 20%)"]).reset_index(drop=True)
EMPIRE_capacity_ow

,Node,GeneratorTechnology,generatorMaxInstallCapacity in MW
0,Belgium,Wind_offshr_grounded,0.00
1,Germany,Wind_offshr_grounded,0.00
2,Denmark,Wind_offshr_grounded,0.00
3,Estonia,Wind_offshr_grounded,1500.00
4,Spain,Wind_offshr_grounded,13000.00
5,Finland,Wind_offshr_grounded,15500.00
6,France,Wind_offshr_grounded,57900.00
7,Great Brit.,Wind_offshr_grounded,0.00
8,Greece,Wind_offshr_grounded,15300.00
9,Ireland,Wind_offshr_grounded,22200.00


Adjust and change only offshore wind in North Sea

Calculated from 6 MW/km2 based on https://www.sciencedirect.com/science/article/pii/S0360544222025403

In [16]:
EMPIRE_capacity_ow_ns = EMPIRE_capacity_ow.iloc[22:]
EMPIRE_capacity_ow_ns["area_km2"] = EMPIRE_capacity_ow_ns["generatorMaxInstallCapacity  in MW"] / 6
EMPIRE_capacity_ow_ns

/var/folders/rd/zf3y09wj69n3q6493fx5jmtr0000gn/T/ipykernel_75153/2702097068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EMPIRE_capacity_ow_ns["area_km2"] = EMPIRE_capacity_ow_ns["generatorMaxInstallCapacity  in MW"] / 6


,Node,GeneratorTechnology,generatorMaxInstallCapacity in MW,area_km2
22,Moray Firth,Wind_offshr_grounded,6558.00,1093.000000
23,Firth of Forth,Wind_offshr_grounded,12636.00,2106.000000
24,Dogger Bank,Wind_offshr_grounded,19512.00,3252.000000
25,Hornsea,Wind_offshr_grounded,14466.00,2411.000000
26,Outer Dowsing,Wind_offshr_grounded,7518.00,1253.000000
27,Norfolk,Wind_offshr_grounded,9756.00,1626.000000
28,East Anglia,Wind_offshr_grounded,8106.00,1351.000000
29,Borssele,Wind_offshr_grounded,5858.30,976.383333
30,Hollandsee Kust,Wind_offshr_grounded,7997.00,1332.833333
31,Helgoländer Bucht,Wind_offshr_grounded,27492.75,4582.125000


Get areas of NO areas from NVE

In [19]:
NO_areas = gpd.read_file("Data/Havvind_IdentifiserteOmrader_2023.geojson")
NO_areas = NO_areas[["omradeNavn", "anleggType", "omradeAreal_km2"]]
NO_areas

,omradeNavn,anleggType,omradeAreal_km2
0,Nordvest A,Flytende,11307
1,Nordvest C,Flytende,5582
2,Vestavind A,Flytende,1884
3,Sønnavind A,Flytende og bunnfast,2900
4,Sørvest C,Bunnfast,1766
5,Nordvest B,Flytende,3437
6,Vestavind F,Flytende,1989
7,Sørvest E,Flytende og bunnfast,1016
8,Sørvest A,Flytende og bunnfast,1456
9,Vestavind B,Flytende,2985


Handle grounded/floating tech

Sønnavind A more likely only floating; Sørvest A, D & E most likely grounded but floating possible [both incl] (source: NVE details)

In [31]:
NO_areas_tech = NO_areas.copy() 
NO_areas_tech.loc[NO_areas_tech["omradeNavn"] == "Sønnavind A", "anleggType"] = "Flytende"

# Sønnavind only floating
NO_both_techs = NO_areas_tech[NO_areas_tech["anleggType"] == "Flytende og bunnfast"]
both_techs_parks = list(NO_both_techs["omradeNavn"])

# Sørvest A, D & E both techs
df_floating_tech = NO_both_techs.copy()
df_grounded_tech = NO_both_techs.copy()
df_floating_tech["anleggType"] = "Flytende"
df_grounded_tech["anleggType"] = "Bunnfast"
NO_both_techs = pd.concat([df_floating_tech, df_grounded_tech])
NO_both_techs

# Merge together
NO_areas_tech = NO_areas_tech[~NO_areas_tech["omradeNavn"].isin(both_techs_parks)]
NO_areas_tech = pd.concat([NO_areas_tech, NO_both_techs]).reset_index(drop=True)
NO_areas_tech

,omradeNavn,anleggType,omradeAreal_km2
0,Nordvest A,Flytende,11307
1,Nordvest C,Flytende,5582
2,Vestavind A,Flytende,1884
3,Sønnavind A,Flytende,2900
4,Sørvest C,Bunnfast,1766
5,Nordvest B,Flytende,3437
6,Vestavind F,Flytende,1989
7,Vestavind B,Flytende,2985
8,Vestavind C,Flytende,1064
9,Vestavind D,Flytende,734


Change format to EMPIRE

In [32]:
NORWEGIAN_TO_EMPIRE_TECH = dict({
    "Flytende": "Wind_offshr_floating",
    "Bunnfast": "Wind_offshr_grouned"
})

NO_areas_tech["anleggType"] = NO_areas_tech["anleggType"].apply(lambda x: NORWEGIAN_TO_EMPIRE_TECH.get(x))
NO_areas_tech = NO_areas_tech.rename(columns={"omradeNavn": "Node", "omradeAreal_km2": "area_km2", "anleggType": "GeneratorTechnology"})
NO_areas_tech

,Node,GeneratorTechnology,area_km2
0,Nordvest A,Wind_offshr_floating,11307
1,Nordvest C,Wind_offshr_floating,5582
2,Vestavind A,Wind_offshr_floating,1884
3,Sønnavind A,Wind_offshr_floating,2900
4,Sørvest C,Wind_offshr_grouned,1766
5,Nordvest B,Wind_offshr_floating,3437
6,Vestavind F,Wind_offshr_floating,1989
7,Vestavind B,Wind_offshr_floating,2985
8,Vestavind C,Wind_offshr_floating,1064
9,Vestavind D,Wind_offshr_floating,734


Merge together existing EMPIRE North Sea areas and NO areas and calculate Max Capacity (MW) based on density capacity from NVE

Source: https://veiledere.nve.no/havvind/identifisering-av-utredningsomrader-for-havvind/metode-og-vurderinger/arealbehov-for-30-gw/

In [34]:
CAPACITY_DENSITY_BASE = 5 # MW/km2
CAPACITY_DENSITY_LOW = 3.5 # MW/km2
CAPACITY_DENSITY_HIGH = 7.5 # MW/km2

In [45]:
OW_areas_ns = pd.concat([EMPIRE_capacity_ow_ns, NO_areas_tech], ignore_index=True)
OW_areas_ns = OW_areas_ns[["Node", "GeneratorTechnology", "area_km2"]]

OW_areas_ns_high = OW_areas_ns.copy()
OW_areas_ns_high["generatorMaxInstallCapacity in MW"] = OW_areas_ns_high["area_km2"] * CAPACITY_DENSITY_HIGH
OW_areas_ns_high

OW_areas_ns_low = OW_areas_ns.copy()
OW_areas_ns_low["generatorMaxInstallCapacity in MW"] = OW_areas_ns_low["area_km2"] * CAPACITY_DENSITY_LOW
OW_areas_ns_low

OW_areas_ns_base = OW_areas_ns.copy()
OW_areas_ns_base["generatorMaxInstallCapacity in MW"] = OW_areas_ns_base["area_km2"] * CAPACITY_DENSITY_BASE
OW_areas_ns_base

,Node,GeneratorTechnology,area_km2,generatorMaxInstallCapacity in MW
0,Moray Firth,Wind_offshr_grounded,1093.000000,5465.000000
1,Firth of Forth,Wind_offshr_grounded,2106.000000,10530.000000
2,Dogger Bank,Wind_offshr_grounded,3252.000000,16260.000000
3,Hornsea,Wind_offshr_grounded,2411.000000,12055.000000
4,Outer Dowsing,Wind_offshr_grounded,1253.000000,6265.000000
5,Norfolk,Wind_offshr_grounded,1626.000000,8130.000000
6,East Anglia,Wind_offshr_grounded,1351.000000,6755.000000
7,Borssele,Wind_offshr_grounded,976.383333,4881.916667
8,Hollandsee Kust,Wind_offshr_grounded,1332.833333,6664.166667
9,Helgoländer Bucht,Wind_offshr_grounded,4582.125000,22910.625000


Export CSV

In [46]:
OW_areas_ns_base.to_csv("Output/offshore_capacities_base.csv", index=False, encoding="utf-8-sig")
OW_areas_ns_low.to_csv("Output/offshore_capacities_low.csv", index=False, encoding="utf-8-sig")
OW_areas_ns_high.to_csv("Output/offshore_capacities_high.csv", index=False, encoding="utf-8-sig")